In [ ]:
import numpy as np


# Module 7: Practical - Normalizing Flows

Normalizing flows solve the problem of evaluating the likelyhood function by modeling the mapping from the latent distribution using a form, whose likelihood and parameters are tractable to calculate.

$z = f(x)$

$p_{data}(x) = p_{latent}(z)|J(x)|$

$Loss$ =

$-\mathbb{E}_{x \sim p_{data}} (\log(p_{data}(x))) =$

$-\sum_{i=1}^N \log( p_{model}(x_i | w ) = -\sum_{i=1}^N \log\Big( p_{latent}(z_i | w ) |J(x_i)|\Big) =$

$-\sum_{i=1}^N \Big( \log( p_{latent}(f(x_i | w )) + \log(|J(x_i)|) \Big)$

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import re

# Load and preprocess Count of Monte Cristo
url = "https://www.gutenberg.org/cache/epub/1184/pg1184.txt"

import requests
text = requests.get(url).text

# Keep only the main body (remove header/footer)
start_idx = text.find("Chapter 1.")
end_idx = text.rfind("Chapter 5.") # text.rfind("End of the Project Gutenberg")
text = text[start_idx:end_idx]

# Pre-processing
text = re.sub(r"[^a-zA-Z0-9\s]", "", text)
text = text.lower()

# Tokenization
tokens = text.split()

# Vocabulary construction
from collections import Counter
counter = Counter(tokens)

# We'll assign indices 0 and 1 to special tokens "<PAD>" and "<UNK>", the rest of the indeces
# are based on the frequency of the words.
vocab = {word: idx+2 for idx, (word, _) in enumerate(counter.most_common(9998))}
vocab["<PAD>"] = 0
vocab["<UNK>"] = 1
inv_vocab = {idx: word for word, idx in vocab.items()}

# Encode tokens
encoded = [vocab.get(word, vocab["<UNK>"]) for word in tokens]


Since we are training the model to predict the next word in a sequence, we will construct our training set features based on 30 word sequences from the text. The corresponding labels are the sequences shifted by one word.

In [ ]:
# Create sequences
SEQ_LEN = 30
class TextDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data) - SEQ_LEN

    def __getitem__(self, idx):
        return (torch.tensor(self.data[idx:idx+SEQ_LEN]),
                torch.tensor(self.data[idx+1:idx+SEQ_LEN+1]))

train_datasets = TextDataset(encoded)
train_loader = DataLoader(train_datasets, batch_size=64, shuffle=True)


Let's see what the first pair of input/output sequences look like.

In [ ]:
next(iter(train_loader))


We now define the LSTM architecture. Since the LSTM layer is already implemented in PyTorch, we can use it directly. The LSTM layer takes the input sequence and returns the output sequence along with the hidden state. The output is then passed through a fully connected layer to get the final predictions.

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, vocab_size=10000, embedding_dim=100, hidden_dim=128):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, hidden=None):
        x = self.embedding(x)
        x, hidden = self.lstm(x, hidden)
        x = self.fc(x)
        return x, hidden

model = LSTMModel()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())


Finally, we train the model for 15 epochs. The loss should decrease over time, indicating that the model is learning to predict the next word in the sequence. After training, we can generate text by providing a seed phrase and letting the model predict the next words based on the learned patterns.

Finally we generate new text by feeding a seed sentence to the model. We then

In [ ]:
def generate_text(model, seed_text, length=50, temperature=1.0):
    model.eval()
    words = seed_text.lower().split()
    input_ids = [vocab.get(w, vocab["<UNK>"]) for w in words]
    input_tensor = torch.tensor(input_ids).unsqueeze(0)
    hidden = None

    with torch.no_grad():
        for _ in range(length):
            output, hidden = model(input_tensor, hidden)
            logits = output[0, -1] / temperature
            probs = torch.nn.functional.softmax(logits, dim=-1)
            next_id = torch.multinomial(probs, num_samples=1).item()
            words.append(inv_vocab.get(next_id, "<UNK>"))

            # Extend input sequence with new token
            input_ids.append(next_id)
            input_tensor = torch.tensor(input_ids).unsqueeze(0)

    return " ".join(words)


In [ ]:
seed = "the count of monte cristo"
print("\nGenerated Text:\n")
print(generate_text(model, seed, length=50))
